Originalement BERT a été pré-entrainé sur des datasets Wikipedia et BookCorpus (dispo sur hugging face)

*Note: For wikipedia we will use the 20220301, which is different from the original split. As a first step we are loading the datasets and merging them together to create on big dataset.*

In [1]:
from datasets import load_dataset, concatenate_datasets

bookcorpus = load_dataset("bookcorpus", split="train")
wiki = load_dataset("wikipedia", "20220301.en", split="train")
wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [2]:
assert bookcorpus.features.type == wiki.features.type
raw_datasets = concatenate_datasets([bookcorpus, wiki])

In [4]:
raw_datasets[:10]["text"]

['usually , he would be tearing around the living room , playing with his toys .',
 'but just one look at a minion sent him practically catatonic .',
 "that had been megan 's plan when she got him dressed earlier .",
 "he 'd seen the movie almost by mistake , considering he was a little young for the pg cartoon , but with older cousins , along with her brothers , mason was often exposed to things that were older .",
 'she liked to think being surrounded by adults and older kids was one reason why he was a such a good talker for his age .',
 "`` are n't you being a good boy ? ''",
 'she said .',
 'mason barely acknowledged her .',
 'instead , his baby blues remained focused on the television .',
 'since the movie was almost over , megan knew she better slip into the bedroom and finish getting ready .']

In [4]:
from tqdm import tqdm

batch_size = 1_000_000
tasks = []
for i in range(0, len(raw_datasets), batch_size):
    tasks.append((i, min(i + batch_size, len(raw_datasets) + 1) - 1))

def write_sub_file(i_begin, i_end):
    with open(f"./data/text_{i_begin // (i_end - i_begin)}.txt", "w", encoding="utf-8") as f:
        for i in tqdm(range(i_begin, i_end)):
            f.write(raw_datasets[i]["text"] + "\n")

In [ ]:
from multiprocessing import Pool

PARALLEL = True

if PARALLEL:
    with Pool(processes=8, maxtasksperchild=1) as pool:
        pool.starmap(write_sub_file, tasks, chunksize=1)
else:
    res = list(itt.starmap(write_sub_file, tasks))

100%|██████████| 462898/462898 [00:35<00:00, 13080.45it/s]


TODO : advanced dataset preparation (de-duplication, filtering, other preprocessing tasks ...)